In [61]:
import xarray as xr
import numpy as np
import pandas as pd

In [70]:
ds = xr.Dataset(data_vars={"tmax":[],
                           "tmin":[]},
                coords={"sa1": [],
                        "day": []})

ds

<xarray.Dataset>
Dimensions:  (tmax: 0, tmin: 0, sa1: 0, day: 0)
Coordinates:
  * tmax     (tmax) float64 
  * tmin     (tmin) float64 
  * sa1      (sa1) float64 
  * day      (day) float64 
Data variables:
    *empty*

In [68]:
# Create an empty DataArray with dimensions for SA1, time, and temperature (min and max)
data_cube = xr.DataArray(
    coords={"SA1": [], "time": [], "temperature": ["min_temperature", "max_temperature"]},
    dims=["SA1", "time", "temperature"],
)

In [47]:
def append_temperature_data(dc, sa1, date, min_temperature, max_temperature):
    new_data = xr.DataArray(
        np.array([[[min_temperature, max_temperature]]]),
        coords={"SA1": [sa1], "time": [date], "temperature": ["min_temperature", "max_temperature"]},
        dims=["SA1", "time", "temperature"],
    )

    if sa1 not in dc.SA1:
        dc = xr.concat([dc, new_data], dim="SA1")
    elif date not in dc.time:
        dc = xr.concat([dc, new_data], dim="time")
    else:
        dc.loc[sa1, date, :] = [min_temperature, max_temperature]

    return dc

In [48]:
# Example usage of the append_temperature_data function
data_cube = append_temperature_data(data_cube, 'SA1_1', '2011-01-01', 10.5, 25.3)
# data_cube = append_temperature_data(data_cube, 'SA1_2', '2011-01-02', 12.3, 22.7)
# data_cube = append_temperature_data(data_cube, 'SA1_3', '2011-01-03', 8.9, 21.5)

C:\Users\haoc4\Documents\AURIN\AusUrbHI\_ausurbhi_venv\Lib\site-packages\xarray\core\dataarray.py:854: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return key in self.data


In [49]:
data_cube

<xarray.DataArray (SA1: 1, time: 1, temperature: 2)>
array([[[10.5, 25.3]]])
Coordinates:
  * time         (time) <U32 '2011-01-01'
  * temperature  (temperature) <U15 'min_temperature' 'max_temperature'
  * SA1          (SA1) <U32 'SA1_1'

In [50]:
def query_temperature_data(data_cube, sa1, date):
    temperature_data = data_cube.sel(SA1=sa1, time=date)
    return temperature_data

In [51]:
temperature_data = query_temperature_data(data_cube, 'SA1_1', '2011-01-01')
print(temperature_data)

<xarray.DataArray (temperature: 2)>
array([10.5, 25.3])
Coordinates:
    time         <U32 '2011-01-01'
  * temperature  (temperature) <U15 'min_temperature' 'max_temperature'
    SA1          <U32 'SA1_1'


In [52]:
data_cube.to_netcdf("temperature_data_cube.nc")

In [53]:
# use open_dataarray() for a single array dataset instead
data_cube = xr.open_dataset("C:\\Users\\haoc4\\Documents\\AURIN\\AusUrbHI\\excess_heat_factor\\ehfheatwaves\\tests\\climpact2.sampledata.gridded.1991-2010.nc")

In [54]:
data_cube

<xarray.Dataset>
Dimensions:    (lon: 24, lat: 19, time: 7305, nb2: 2)
Coordinates:
  * lon        (lon) float32 110.6 112.5 114.4 116.2 ... 148.1 150.0 151.9 153.8
  * lat        (lat) float32 -10.48 -12.38 -14.29 ... -40.95 -42.86 -44.76
  * time       (time) datetime64[ns] 1991-01-01 1991-01-02 ... 2010-12-31
Dimensions without coordinates: nb2
Data variables:
    time_bnds  (time, nb2) datetime64[ns] ...
    tmax       (time, lat, lon) float32 ...
    tmin       (time, lat, lon) float32 ...
    precip     (time, lat, lon) float32 ...
Attributes:
    history:  Wed Jun 17 11:12:30 2015: ncatted -a ,global,d,, climpact2.samp...

In [66]:
# Create empty DataArrays for tmax and tmin
tmax = xr.DataArray(np.empty((0, 0)), coords=[[], []], dims=['date', 'sa1'], name='tmax')
tmin = xr.DataArray(np.empty((0, 0)), coords=[[], []], dims=['date', 'sa1'], name='tmin')

# Initialize empty values with NaN
tmax[:] = np.nan
tmin[:] = np.nan

# Create the Dataset
ds = xr.Dataset({'tmax': tmax, 'tmin': tmin})

print(ds)

<xarray.Dataset>
Dimensions:  (date: 0, sa1: 0)
Coordinates:
  * date     (date) float64 
  * sa1      (sa1) float64 
Data variables:
    tmax     (date, sa1) float64 
    tmin     (date, sa1) float64 
